In [85]:
from datetime import datetime, timedelta

import pandas as pd
import glob
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib.dates import DateFormatter
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [86]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [87]:
path = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/'
RootFilePath = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/Layer1Data/Output/'

In [88]:
# Reading the file
channelData = pd.concat(map(lambda file: pd.read_csv(file, header = 0 , sep ='\t', index_col=0), glob.glob(os.path.join('', path+'Layer1Data/*.csv'))))

C:\Users\jainsac\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [89]:
channelData['scan_time'] = pd.to_datetime(channelData['scan_time'])

In [90]:
channel_list = channelData.channel_id.unique().tolist()

In [91]:
# List provided by Sami
channel_list

[56334, 56340, 56341, 56342, 56343]

In [92]:
#path = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/'

# Reading all the channel data files and loading
# them in channelData dataframe. channelData data
# frame contains following attributes: scan_time, 
# channel_id, power_dbm
#allFiles = glob.glob(path+'/channels*.p')
#channelData = pd.concat((pickle.load(open(f, "rb")) for f in allFiles))

year = 2016
month = 10

# Reading the timerange file and loading
# them in timeRange data frame
ranges = glob.glob(path+'/time*.p')
timeRange = pd.concat((pickle.load(open(f, "rb")) for f in ranges))

In [93]:
dts = [dt.strftime('%Y-%m-%d %H:%M:%S.%f ') for dt in 
       datetime_range(datetime(year, month, 1, 0, 0, 0, 0), datetime(year, month, 31, 0, 0, 0, 0), 
                      timedelta(minutes=30))]

In [94]:
# code below takes the subset of the data from channel and
# time range and aggregate the value of occupancy for each time interval
# in the end list of occupancy and start time values are saved into temp_df dataframe
for ch in channel_list:
    #print('Channel: '+str(ch))
    i = 0
    j = 1
    a_list = []
    b_list = []
    end_list = []
    while (i < len(dts) and j < len(dts)):
        channel = ch
        startTime = dts[i]
        endTime = dts[j]
        #print('Value of i: '+str(i))
        #print('Value of j: '+str(j))

        #print('Start Time:' +startTime)
        #print('End Time: '+ endTime)
        chanMask = channelData.channel_id == channel
        #print('Chanmask')
        #print(chanMask.head())
        
        #print('Timemask')
        timeMask = (channelData.scan_time > startTime) & (channelData.scan_time <= endTime)
        #print(timeMask.head())
        
        #print('Subset')
        subset = channelData.loc[chanMask & timeMask].reset_index(drop = True)
        #print(subset.head())
        #print('Subset length: '+str(ch)+': '+str(len(subset)))

        timeMask2 = (timeRange.scan_time > startTime) & (timeRange.scan_time <= endTime)
        subTimeRange = timeRange.loc[timeMask2].reset_index(drop = True)
        #print(subTimeRange.head())
        #print('Subtime Range length: '+str(ch)+': '+str(len(subTimeRange)))

        if (len(subset) and len(subTimeRange)) > 0:
            merged = pd.merge(left = subTimeRange, right = subset, how = 'left')
            merged = merged.fillna(value = {'channel_id': channel})
            
            #condition: pd.isnull(merged.power_dbm)
            #when condition is true value is 1 else 0
            merged['isEmpty'] = np.where(pd.isnull(merged.power_dbm), 1, 0)
            
            #print(1-sum(merged.isEmpty))
            #print(merged.shape[0])
            
            OccPercent = 1-sum(merged.isEmpty)/merged.shape[0]
            a_list.append(startTime)
            b_list.append(OccPercent*100)
            end_list.append(endTime)
        i = i+1
        j = j+1

    temp_df = pd.DataFrame({'Channel': ch, 'StartTime': a_list, 'EndTime': end_list, 'Occupancy': b_list}, columns=['StartTime', 'EndTime', 'Channel', 'Occupancy']) 
    #writer = pd.ExcelWriter(RootFilePath+'30MinOutput'+str(ch)+'.xlsx')
#    temp_df.to_excel(writer,'Sheet1', index=False)
    temp_df.to_csv(RootFilePath + str(ch)+'.csv', sep='\t', index=False)
    #writer.save()